In [14]:
import pandas as pd
import numpy as np

# makelaar data: 
makelaar_path = r'C:\Users\bgriffioen\OneDrive - STX Commodities B.V\Desktop\funda-project\funda-tool\data\makelaar\makelaar_results_2025-06-23.csv'
funda_path = r'C:\Users\bgriffioen\OneDrive - STX Commodities B.V\Desktop\funda-project\funda-tool\data\funda\funda_data_2025-06-23.csv'
makelaar_df = pd.read_csv(makelaar_path, sep=",")
funda_df = pd.read_csv(funda_path, sep=",")
path = r"C:\Users\bgriffioen\OneDrive - STX Commodities B.V\Desktop\funda-project\funda-tool\data\funda_makelaars_combined_2025-06-23.csv"
combined_df = pd.read_csv(path, sep=",")

In [35]:
# df if m2 = 1 convert to nan
combined_df.loc[combined_df['m2'] == 1, 'm2'] = np.nan
combined_df['price/m2'] = combined_df['price'] / combined_df['m2']

# print( number of funda listings:)
# funda source == 'sourc
# Print number of funda listings
funda_listings = combined_df[combined_df['source'].isin(['funda', 'both'])]
print("Number of funda listings:", len(funda_listings))

makelaar_listings = combined_df[combined_df['source'].isin(['makelaar', 'both'])]
print("Number of makelaar listings:", len(makelaar_listings))

Number of funda listings: 569
Number of makelaar listings: 524


In [ ]:

# makelaar_df street, number_extension,latitude, longitude,url
makelaar_df = makelaar_df[['street', 'number_extension', 'latitude', 'longitude', 'url']]
# funda_df street_name, number url ,lat, lon
funda_df = funda_df[['street_name', 'number', 'url', 'lat', 'lon']]

# number_no_extension, take only the first part of the street name
makelaar_df['number_clean'] = makelaar_df['number_extension'].str.split(' ').str[0]
funda_df['number_clean'] = funda_df['number'].astype(str).str.split(' ').str[0]

# how many listings in makelaar_df are also in funda_df 
# 1. Based on street and number
makelaar_df['street_number'] = makelaar_df['street'] + ' ' + makelaar_df['number_clean'].fillna('')
funda_df['street_number'] = funda_df['street_name'] + ' ' + funda_df['number_clean'].astype(str)
common_listings = makelaar_df[makelaar_df['street_number'].isin(funda_df['street_number'])]
print("Number of common listings based on street and number:", len(common_listings))



Number of common listings based on street and number: 45
Number of common listings based on latitude and longitude: 1656


In [ ]:
# Drop rows with missing coordinates
makelaar_coords = makelaar_df.dropna(subset=['latitude', 'longitude']).copy()
funda_coords = funda_df.dropna(subset=['lat', 'lon']).copy()

# Perform a cross join using pandas merge with 'how="cross"'
cross = makelaar_coords.merge(funda_coords, how='cross', suffixes=('_makelaar', '_funda'))

# Filter by latitude and longitude difference within tolerance
tolerance = 0.0001  # Define a tolerance for latitude and longitude comparison
common = cross[
    (np.abs(cross['latitude'] - cross['lat']) < tolerance) &
    (np.abs(cross['longitude'] - cross['lon']) < tolerance)
]

print("Number of common listings based on latitude and longitude:", len(common))

Number of common listings based on latitude and longitude: 164


In [32]:
# Cross join funda_coords to makelaar_coords
cross_funda_to_makelaar = funda_coords.merge(makelaar_coords, how='cross', suffixes=('_funda', '_makelaar'))

# Filter by latitude and longitude difference within tolerance
common_funda_to_makelaar = cross_funda_to_makelaar[
    (np.abs(cross_funda_to_makelaar['lat'] - cross_funda_to_makelaar['latitude']) < tolerance) &
    (np.abs(cross_funda_to_makelaar['lon'] - cross_funda_to_makelaar['longitude']) < tolerance)
]

print("Number of funda listings also in makelaar listings (lat/lon):", len(common_funda_to_makelaar))


Number of funda listings also in makelaar listings (lat/lon): 164
